In [2]:
import json
import pandas as pd
import numpy as np

In [3]:
logs = []
for i in range(1,9):
    with open(f"Tablet/log_id_{i}.json") as f:
        log = json.load(f)
        for l in log:
            logs.append(l)

with open(f"Tablet/log_id_video.json") as f:
    log = json.load(f)
    for l in log:
        logs.append(l)
df = pd.DataFrame(logs)

In [4]:
df["environment"] = "tablet"
df.to_csv('Tablet/userTestEvents.csv', index=False)

In [5]:
# df["event"].count()
# df[df["event"] == "void_selection"]
# df["id"] = np.where(df["id"] == "10", 0, df["id"])
df["id"] = df["id"].where(df["id"] != "11", 0)

In [6]:
# logs = []
# for i in range(1,9):
#     with open(f"webfiles/reqs/RAreq{i}.json") as f:
#         log = json.load(f)
#         for l in log:
#             logs.append(l)
# dfr = pd.DataFrame(logs)

In [7]:
# dfr.to_csv('chartReqs.csv', index=False)

In [8]:
# endTime = df[(df["event"] == "start_evaluation") & (df["task"] == 1)]["timestamp"].values # no start enough
# startTime = df[(df["task"] == 1)].groupby("id").first()["timestamp"].values
# endTime = df[(df["task"] == 2)].groupby("id").first()["timestamp"].values

# finalTime = df[(df["event"] == "finish_task") & (df["task"] == 5)]["timestamp"].values
# totalTimes = (finalTime - startTime) / 1000
# totalTimes = (endTime - startTime) / 1000
# totalTimes

In [9]:
# "ID_PARTICIPANTE,TASK_NUMBER,TASK_COMPLETION_TIME,NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK"
ids = df["id"].unique()

dfs = pd.DataFrame({})
for task in range(2,6):        
    startTime = df[(df["task"] == task-1)].groupby("id").last()["timestamp"].values
    endTime = df[(df["task"] == task)].groupby("id").last()["timestamp"].values
    taskTimesSeconds = (endTime - startTime) / 1000
    clicks = df[(df["task"] == task)].groupby("id")["event"].count().values
    dfs = pd.concat([dfs, pd.DataFrame({"ID_PARTICIPANTE": ids, "TASK_NUMBER": task, "TASK_COMPLETION_TIME": taskTimesSeconds,
                                       "NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK":clicks})])

extra_task = 1
startTime = df[(df["task"] == extra_task)].groupby("id").first()["timestamp"].values
endTime = df[(df["task"] == extra_task)].groupby("id").last()["timestamp"].values
taskTimesSeconds = (endTime - startTime) / 1000
clicks = df[(df["task"] == 1)].groupby("id")["event"].count().values
dfs = pd.concat([dfs, pd.DataFrame({"ID_PARTICIPANTE": ids, "TASK_NUMBER": extra_task, "TASK_COMPLETION_TIME": taskTimesSeconds,
                                   "NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK":clicks})])
dfs["environment"] = "tablet"

In [10]:
# dfs

In [11]:
dfs.to_csv('Tablet/userTestMonitoringTablet.csv', index=False)

In [12]:
#df["event"].unique()

In [13]:
dfar = pd.read_csv("AR/userTestEvents.txt")
dfar["id"] = dfar["ID_PARTICIPANTE"]
dfar["task"] = dfar["TASK_NUMBER"]

dfar["TIME_OF_EVENT"] = dfar["TIME_OF_EVENT"].apply(lambda a: a[len("738579."):-len("7230")])
dfar["TIME_OF_EVENT"] = np.where((dfar["id"] == "1") | (dfar["id"] == "2"),
                                 dfar["TIME_OF_EVENT"].apply(lambda a: "2023-03-09 " + a),
                                 dfar["TIME_OF_EVENT"].apply(lambda a: "2023-03-10 " + a))
dfar["TIME_OF_EVENT"] = pd.to_datetime(dfar["TIME_OF_EVENT"])
dfar = dfar.rename(columns={"EVENT": "fullEvent", "TIME_OF_EVENT": "timestamp"})
dfar

,ID_PARTICIPANTE,TASK_NUMBER,timestamp,TASK_CLICK_NUMBER,fullEvent,id,task
0,0,1,2023-03-10 19:09:33.732,1,Open: 260,0,1
1,0,1,2023-03-10 19:09:35.606,2,PersonalInfo/GeneralStationInfo/Buttons/CycleT...,0,1
2,0,1,2023-03-10 19:09:37.340,3,PersonalInfo/CycleTimesMain/TitleBar/Buttons/B...,0,1
3,0,2,2023-03-10 19:09:40.271,1,HandMenu/LineMonitoringMenu/UsabilityTaskBar/B...,0,2
4,0,2,2023-03-10 19:09:42.119,2,PersonalInfo/GeneralStationInfo/Buttons/KPIsBu...,0,2
...,...,...,...,...,...,...,...
275,8,5,2023-03-10 16:14:56.452,4,PersonalInfo/GeneralStationInfo/Buttons/KPIsBu...,8,5
276,8,5,2023-03-10 16:14:58.075,5,PersonalInfo/KPIsMain/Buttons/PartCountButton,8,5
277,8,5,2023-03-10 16:15:08.476,6,PersonalInfo/KPIsSpecific/ShiftNumberSlider,8,5
278,8,5,2023-03-10 16:15:24.814,7,PersonalInfo/KPIsSpecific/TitleBar/Buttons/Clo...,8,5


In [14]:
with open(f"AR/eventsARList.json") as f:
    arevents = json.load(f)

In [15]:
# day = pd.to_datetime(df[df["id"] != 0]["timestamp"],unit='ms')
# day
# pd.concat([df,day],axis=1).groupby("id").first()

In [16]:
subkeys = ["event", "context", "subcontext", "attr"]
for key in list(arevents.keys())[1:]:
    are = arevents[key]
    evs = [are.get(sub, None) for sub in subkeys]
    dfar.loc[dfar["fullEvent"] == key, subkeys] = evs

for station in ["260", "270", "290"]:
    dfar.loc[dfar["fullEvent"] == f"Open: {station}", ["event", "station"]] = ["select_station", station] 
dfar = dfar.drop(['ID_PARTICIPANTE', 'TASK_NUMBER'], axis=1)
dfar["environment"] = "ar"

In [17]:
dfar.to_csv('AR/userTestEvents.csv', index=False)

In [21]:
dfmon = pd.read_csv("AR/userTestMonitoringAR.txt")
dfmon["TASK_COMPLETION_TIME"] = dfmon["TASK_COMPLETION_TIME"].apply(lambda a: 60*float(a[len("00:"):len("00:")+2]) + float(a[len("00:00:"):]))
dfmon["environment"] = "ar"
dfmon.to_csv("AR/userTestMonitoringAR.csv", index=False)

In [36]:
# task=4
# part="3"
# env="ar"
# print(task,part,env)
# c = (dfar["task"] == task) & (dfar["environment"] == env) & (dfar["id"] == part)

# dfar[c]["timestamp"] - dfar[c]["timestamp"].shift()

4 3 ar


90                       NaT
91    0 days 00:00:07.452000
92    0 days 00:00:17.592000
93    0 days 00:00:08.938000
94    0 days 00:00:10.967000
95    0 days 00:00:01.907000
96    0 days 00:00:03.538000
97    0 days 00:00:00.615000
98    0 days 00:00:00.214000
99    0 days 00:00:01.185000
100   0 days 00:00:01.899000
101   0 days 00:00:02.688000
102   0 days 00:00:04.166000
103   0 days 00:00:05.064000
104   0 days 00:00:34.117000
105   0 days 00:00:01.917000
106   0 days 00:00:01.713000
Name: timestamp, dtype: timedelta64[ns]

In [41]:
finalMon = pd.concat([dfmon, dfs])
finalMon.to_csv("datasets/userTestMonitoring.csv", index=False)

df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')
df["id"] = df["id"].apply(lambda a: str(a))
dfar["id"] = dfar["id"].apply(lambda a: str(a))
finalTest = pd.concat([dfar, df])

finalTest["time"] = finalTest["timestamp"].values.astype(np.int64) / 10 ** 9. ## // 10 ** 9
for task in range(1,6):
    for env in ["ar", "tablet"]:
        for part in finalTest["id"].unique():
            cond = (finalTest["task"] == task) & (finalTest["environment"] == env) & (finalTest["id"] == str(part)) 
            if (len(finalTest[cond]["time"].values) > 0):
                # print(finalTest[cond]["time"] - finalTest[cond]["time"].shift(), env)
                finalTest.loc[cond,"time"] = (finalTest[cond]["time"] - finalTest[cond]["time"].values[0]).astype(float)
                finalTest.loc[cond,"time_shift"] = finalTest[cond]["time"] - finalTest[cond]["time"].shift()
            else:
                finalTest.loc[cond,"time"] = 0
                finalTest.loc[cond,"time_shift"] = 0

finalTest.to_csv('datasets/userTestEvents.csv', index=False)